In [ ]:
!pip install --upgrade langchain_openai
!pip install --upgrade langchain
!pip install --upgrade pandas
!pip install --upgrade langchain_experimental

In [ ]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

In [ ]:
import pandas as pd
from langchain_openai import OpenAI

In [ ]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv"
)

In [ ]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
pandas_agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-4o"),
    df,
    verbose=True,
    allow_dangerous_code=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [ ]:
pandas_agent.invoke("how many rows are there?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'len(df)'}`


891The dataframe contains 891 rows.

> Finished chain.


{'input': 'how many rows are there?',
 'output': 'The dataframe contains 891 rows.'}

In [ ]:
from langchain_core.tools import StructuredTool

In [ ]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.tools import tool

In [ ]:
# analysis_tool = StructuredTool.from_function(
#     func=pandas_agent.invoke,
#     name="Analysis tool",
#     description="Uses pandas agent to analyze the data",
#     # args_schema=CalculatorInput,
#     return_direct=True,
#     # coroutine= ... <- you can specify an async method if desired as well
# )

In [ ]:
@tool
def analysis_tool(query: str) -> str:
    """
    Uses pandas agent to analyze the data.
    """
    return pandas_agent.invoke(query)

In [ ]:
from typing import List, Dict, Any

@tool
def analysis_tool(query: str, json_data: List[Dict[str, Any]]) -> str:
    """
    Uses pandas agent to analyze the data.
    Should take in query string and also the full json data as 'json_data'
    """
    df = pd.DataFrame(json_data)
    pandas_agent = create_pandas_dataframe_agent(
      ChatOpenAI(temperature=0, model="gpt-4o"),
      df,
      verbose=True,
      allow_dangerous_code=True,
      agent_type=AgentType.OPENAI_FUNCTIONS,
    )
    return pandas_agent.invoke(query)

In [ ]:
import requests
from typing import List, Dict, Any

@tool
def get_todos() -> List[Dict[str, Any]]:
    """
    Fetch all todos from the API.
    Returns a list of dictionaries containing todo items.
    """
    response = requests.get("https://jsonplaceholder.typicode.com/todos")
    response.raise_for_status()
    return response.json()

@tool
def get_posts() -> List[Dict[str, Any]]:
    """
    Fetch all posts from the API.
    Returns a list of dictionaries containing post items.
    """
    response = requests.get("https://jsonplaceholder.typicode.com/posts")
    response.raise_for_status()
    return response.json()

@tool
def get_comments() -> List[Dict[str, Any]]:
    """
    Fetch all comments from the API.
    Returns a list of dictionaries containing comment items.
    """
    response = requests.get("https://jsonplaceholder.typicode.com/comments")
    response.raise_for_status()
    return response.json()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an LLM powered agent which analyzes data on behalf of a user within a blogging platform. You have access to their posts, comments, and todos via api connector tools. You also have a tool which you can use to analyze the response data. If you use the analysis tool, pass it both a query and the json reponse from the api tool as json_data."),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [ ]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [analysis_tool, get_todos, get_posts, get_comments]

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "how many todos do I have?"})



> Entering new AgentExecutor chain...

Invoking: `get_todos` with `{}`


[{'userId': 1, 'id': 1, 'title': 'delectus aut autem', 'completed': False}, {'userId': 1, 'id': 2, 'title': 'quis ut nam facilis et officia qui', 'completed': False}, {'userId': 1, 'id': 3, 'title': 'fugiat veniam minus', 'completed': False}, {'userId': 1, 'id': 4, 'title': 'et porro tempora', 'completed': True}, {'userId': 1, 'id': 5, 'title': 'laboriosam mollitia et enim quasi adipisci quia provident illum', 'completed': False}, {'userId': 1, 'id': 6, 'title': 'qui ullam ratione quibusdam voluptatem quia omnis', 'completed': False}, {'userId': 1, 'id': 7, 'title': 'illo expedita consequatur quia in', 'completed': False}, {'userId': 1, 'id': 8, 'title': 'quo adipisci enim quam ut ab', 'completed': True}, {'userId': 1, 'id': 9, 'title': 'molestiae perspiciatis ipsa', 'completed': False}, {'userId': 1, 'id': 10, 'title': 'illo est ratione doloremque quia maiores aut', 'completed': True}, {'userId': 1, 'id': 11, '

{'input': 'how many todos do I have?',
 'output': 'You have a total of 200 todos.'}

In [ ]:
agent_executor.invoke({"input": "what percent of my todos are completed?"})



> Entering new AgentExecutor chain...

Invoking: `get_todos` with `{}`


[{'userId': 1, 'id': 1, 'title': 'delectus aut autem', 'completed': False}, {'userId': 1, 'id': 2, 'title': 'quis ut nam facilis et officia qui', 'completed': False}, {'userId': 1, 'id': 3, 'title': 'fugiat veniam minus', 'completed': False}, {'userId': 1, 'id': 4, 'title': 'et porro tempora', 'completed': True}, {'userId': 1, 'id': 5, 'title': 'laboriosam mollitia et enim quasi adipisci quia provident illum', 'completed': False}, {'userId': 1, 'id': 6, 'title': 'qui ullam ratione quibusdam voluptatem quia omnis', 'completed': False}, {'userId': 1, 'id': 7, 'title': 'illo expedita consequatur quia in', 'completed': False}, {'userId': 1, 'id': 8, 'title': 'quo adipisci enim quam ut ab', 'completed': True}, {'userId': 1, 'id': 9, 'title': 'molestiae perspiciatis ipsa', 'completed': False}, {'userId': 1, 'id': 10, 'title': 'illo est ratione doloremque quia maiores aut', 'completed': True}, {'userId': 1, 'id': 11, '

{'input': 'what percent of my todos are completed?',
 'output': '45.0% of your todos are completed.'}

### Testing plan
- Generate 5 questions for each endpoint.
- Go through and populate correct answer.
- Set up LLM as judge to determine if answer is correct.
- Loop through and see accuracy.

In [ ]:
test_set = [
    ["Q1", "todos", "What percentage of all todos are completed?"],
    ["Q2", "todos", "Which user has the most incomplete todos?"],
    ["Q3", "todos", "How many users have more than 10 completed todos?"],
    ["Q4", "todos", "What is the average number of todos per user?"],
    ["Q5", "todos", "List the top 3 users by total number of todos assigned to them"],
    ["Q6", "posts", "How many total posts are there in the system?"],
    ["Q7", "posts", "Who are the top 3 users by number of posts?"],
    ["Q8", "posts", "What is the average length of post titles?"],
    ["Q9", "posts", "How many users have written at least one post?"],
    ["Q10", "posts", "Which user has the longest average post body length?"],
    ["Q11", "comments", "What is the average number of comments per post?"],
    ["Q12", "comments", "Which post ID has received the most comments?"],
    ["Q13", "comments", "How many unique email addresses have left comments?"],
    ["Q14", "comments", "What is the total number of comments in the system?"],
    ["Q15", "comments", "What is the average length of comment bodies?"]
]

In [ ]:
ground_truth = [
    ["Q1", "45% of todos are completed"],
    ["Q2", "User 4 has the most incomplete todos with 15 incomplete tasks"],
    # ... and so on
]

In [ ]:
from openai import OpenAI
from typing import Dict, List

In [ ]:
comparison_schema = {
    "type": "object",
    "properties": {
        "reasoning": {"type": "string"},
        "is_correct": {"type": "boolean"}
    },
    "required": ["reasoning", "is_correct"]
}

In [ ]:
async def evaluate_answer(agent_answer: str, ground_truth: str) -> Dict:
    client = OpenAI()
    response = await client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a judge evaluating if two answers are semantically equivalent. First provide some reasoning as to whether the two are the same. Then label the answer as correct or not."},
            {"role": "user", "content": f"Compare these answers:\nAgent's answer: {agent_answer}\nGround truth: {ground_truth}"}
        ],
        response_format={"type": "json_schema", "schema": comparison_schema}
    )
    return response.choices[0].message.content

In [ ]:
# Main evaluation loop
results = []
for test_id, endpoint, question in test_set:
    # Get agent's answer
    agent_answer = agent_executor.invoke({"input": question})

    # Find corresponding ground truth
    truth = next(item[1] for item in ground_truth if item[0] == test_id)

    # Evaluate using structured output
    evaluation = evaluate_answer(agent_answer, truth)

    results.append({
        "id": test_id,
        "question": question,
        "agent_answer": agent_answer,
        "ground_truth": truth,
        "evaluation": evaluation
    })

In [ ]:
import pandas as pd

# After collecting all results
results_df = pd.DataFrame(results)

In [ ]:
# Calculate accuracy metrics
total_questions = len(results_df)
correct_answers = results_df['evaluation'].apply(lambda x: x['is_correct']).sum()
accuracy_percentage = (correct_answers / total_questions) * 100

# Print accuracy metrics
print(f"Accuracy: {accuracy_percentage:.2f}% ({correct_answers}/{total_questions})")

In [ ]:
# Add accuracy by endpoint
accuracy_by_endpoint = results_df.groupby('endpoint').agg({
    'evaluation': lambda x: sum(item['is_correct'] for item in x),
    'id': 'count'
}).rename(columns={'evaluation': 'correct', 'id': 'total'})

accuracy_by_endpoint['accuracy'] = (accuracy_by_endpoint['correct'] / accuracy_by_endpoint['total'] * 100)

# Print detailed breakdown
for idx, row in accuracy_by_endpoint.iterrows():
    print(f"{idx}: {row['accuracy']:.2f}% ({int(row['correct'])}/{int(row['total'])})")

To get ground truth
- Get each one of those fetch all into a CSV
- Feed to julius and have it answer questions